In this project, we are going to analyze the car listing which is belong to Kleinanzeigen, a dataset of used cars from eBay. The data consists of 50.000 rows for each car ad in ebay

With given columns, we will be able to detect the relationship between metrics such as price and vehicle type, gearbox, brand, kilometer etc.

## Data Investigation

In [1]:
import pandas as pn
import numpy as np
autos = pn.read_csv("autos.csv", encoding = "Latin-1")

In [ ]:
autos

In [ ]:
autos.info()
autos.head()

We can see thhat there are both integer and object based column in our database. Considering some of the columns with numerical values, we can edit them and set as integer instead of objects. This will help us to organize and analyze data more easily.

As first modification, we are going to set the column name formats from camelcase to snakecase as preferred column format of python:

## Column Names Modification

In [ ]:
autos.columns #Columns are currently in camelcase format

In [2]:
autos.rename(columns={'yearOfRegistration':'registration_year','monthOfRegistration':'registration_month',
                      'notRepairedDamage':'unrepaired_damage','dateCreated':'ad_created',
                      'dateCrawled':'date_crawled','offerType':'offer_type',
                      'vehicleType':'vehicle_type','powerPS':'power_ps',
                      'fuelType':'fuel_type',
                      'nrOfPictures':'nr_of_pictures','postalCode':'postal_code',
                      'lastSeen':'last_seen'}, inplace = True )

In [ ]:
autos.columns #Columns are updated as snakecase
autos.head()

Now we can see that columns are set as snakecase as we desired.

In [ ]:
autos.describe(include ='all')

* When we analyse the data, we see that some of columns such as "seller", "offer_type" contain mostly 1 value which is privat for "seller" and Angebot for "offer_type".
* It seems nr_of_pictures and postal_code columns have NaN value of unique values which must be examined.
* If we convert columns such as price, odometer by removing sings inside them, we can analyze them better

In [3]:
autos.price = autos.price.str.replace("$","")
autos.price = autos.price.str.replace(",","")
autos["price"] = autos["price"].astype(int)

autos.odometer = autos.odometer.str.replace("km","")
autos.odometer = autos.odometer.str.replace(",","")
autos["odometer"] = autos["odometer"].astype(int)
autos.rename(columns = {"odometer":"odometer_km"},inplace = True)




In [ ]:
autos.info()

We have set the dtypes of price and odometer columns to integer while changing the name of otometer to otometer_km in order to keep the value information

Now lets investigate the outliers of price and odometer_km columns:

In [ ]:
#autos["price"].unique().shape
#autos["price"].describe()
#autos["price"].value_counts().sort_index(ascending = False)

#autos["odometer_km"].unique().shape
#autos["odometer_km"].describe()
#autos["odometer_km"].value_counts().sort_index(ascending = False)

We see that odometer_km seems to have logical values in terms of km. But in price column, there are values that are out of logic for both low values and high values. For that reason, prices which are higher than 1 and lower than 10.000.000 will be set as our new dataset.

In [4]:
autos = autos[autos["price"].between(1,10000000)]

In [ ]:
autos["price"].describe()

## Date Columns

When we analyze our data, there are totally 5 columns that represent date related information. These are:
* `date_crawled`: added by the crawler
* `last_seen`: added by the crawler
* `ad_created`: from the website
* `registration_month`: from the website
* `registration_year`: from the website
Although the last two are in numeric column type, the first three are in string column type. In order to analyze better, we need to modify them and set as numeric as well.

In [ ]:
autos[["date_crawled","last_seen","ad_created","registration_month","registration_year"]]

In [ ]:
#autos["date_crawled"].str[:10].value_counts(normalize = True, dropna = False).sort_index(ascending = True)
#autos["last_seen"].str[:10].value_counts(normalize = True, dropna = False).sort_index(ascending = True)
#autos["ad_created"].str[:10].value_counts(normalize = True, dropna = False).sort_index(ascending = True)
#autos["registration_month"].describe()
#autos["registration_month"].value_counts().sort_index(ascending = True)
autos["registration_year"].describe()

We see that registration_year contains some irrelevant years such as 1000 and 9999 as well as registration_month contains month value 0. Let's check and try to remove rows with unreliable year values if their amount is not exceeding the majority that can affect the quality of our database.

In [5]:
incorrect = (autos["registration_year"] < 1900) | (autos["registration_year"] > 2016)
autos.loc[incorrect,"registration_year"].value_counts().sort_index(ascending = True)

1000       1
1001       1
1111       1
1800       2
2017    1392
2018     470
2019       2
2800       1
4100       1
4500       1
4800       1
5000       4
5911       1
6200       1
8888       1
9000       1
9999       3
Name: registration_year, dtype: int64

We see that 2017 and 2018 years contain much more data than the other outliers. So it would be better remove the outliers excluding 2017 and 2018.

In [6]:
autos = autos[autos["registration_year"].between(1900,2018)]
autos["registration_year"].value_counts(normalize = True).sort_index(ascending = False)

2018    0.009681
2017    0.028671
2016    0.025129
2015    0.008074
2014    0.013656
2013    0.016540
2012    0.026982
2011    0.033429
2010    0.032729
2009    0.042966
2008    0.045623
2007    0.046900
2006    0.055015
2005    0.060474
2004    0.055675
2003    0.055592
2002    0.051205
2001    0.054295
2000    0.065005
1999    0.059691
1998    0.048671
1997    0.040185
1996    0.028280
1995    0.025273
1994    0.012956
1993    0.008754
1992    0.007642
1991    0.006982
1990    0.007147
1989    0.003584
          ...   
1968    0.000536
1967    0.000536
1966    0.000453
1965    0.000350
1964    0.000247
1963    0.000165
1962    0.000082
1961    0.000124
1960    0.000494
1959    0.000124
1958    0.000082
1957    0.000041
1956    0.000082
1955    0.000041
1954    0.000041
1953    0.000021
1952    0.000021
1951    0.000041
1950    0.000062
1948    0.000021
1943    0.000021
1941    0.000041
1939    0.000021
1938    0.000021
1937    0.000082
1934    0.000041
1931    0.000021
1929    0.0000

## Brand Analysis

Now we can start to analyze our database in terms of brand and their distribution among Ebay.

In [7]:
unique_brands_mean_price = {}
unique_brands_max_price = {}

uniques = autos["brand"].unique()
for u in uniques:
    selected_rows = autos[autos["brand"] == u]
    mean_price = selected_rows["price"].mean()
    max_price = selected_rows["price"].max()
    unique_brands_mean_price[u] = mean_price
    unique_brands_max_price[u] = max_price

print(unique_brands_mean_price,"\n")
print(unique_brands_max_price)

{'chrysler': 3519.3846153846152, 'subaru': 3940.470588235294, 'kia': 5889.298550724638, 'bmw': 8493.857251184834, 'fiat': 2793.8700475435817, 'mitsubishi': 3386.911838790932, 'seat': 4320.168661588684, 'volvo': 4866.993166287016, 'jeep': 11590.214953271028, 'mazda': 4059.059539918809, 'suzuki': 4137.807692307692, 'saab': 3183.493670886076, 'peugeot': 3065.611888111888, 'toyota': 5148.0032733224225, 'citroen': 3756.07299270073, 'lancia': 3182.0, 'jaguar': 11525.554054054053, 'smart': 3518.102305475504, 'porsche': 45624.27177700349, 'lada': 2647.7241379310344, 'daihatsu': 1628.4262295081967, 'hyundai': 5371.792960662526, 'volkswagen': 5525.717507016356, 'daewoo': 1079.342105263158, 'land_rover': 18934.272727272728, 'chevrolet': 6620.069090909091, 'ford': 4022.9054093999407, 'dacia': 5897.736434108527, 'mercedes_benz': 8526.623225806452, 'opel': 2941.4664391353813, 'renault': 2431.195698924731, 'trabant': 1789.6268656716418, 'mini': 10541.566985645934, 'skoda': 6353.544871794872, 'rover':

We see that porsche is the brand that has the greatest price average among the others. The lowest price belongs to car brand daewoo.
While checking the maximum prices for each brand, we notice the irrelevant values were belonging to volkswagen (999999) and sonstigue_autos (10000000). So it would have been better if we had deleted these values from our data.

In addition to price, we can analyze average mileage of the cars and try to detect a relationship between price and average mileage column. To do that, combining them in a dataframe will be a useful way for us.

In [8]:
ubmp = pn.Series(unique_brands_mean_price)


unique_brands_avg_mileage = {}

uniques = autos["brand"].unique()
for u in uniques:
    selected_rows = autos[autos["brand"] == u]
    avg_mileage = selected_rows["odometer_km"].mean()
    unique_brands_avg_mileage[u] = avg_mileage
    
ubam = pn.Series(unique_brands_avg_mileage)


combined = pn.DataFrame(ubmp,columns =["mean_price"])
combined["mean_mileage"] = ubam
print(combined)

                  mean_price   mean_mileage
alfa_romeo       4050.725000  130984.375000
audi             9212.930662  129492.562380
bmw              8493.857251  132686.255924
chevrolet        6620.069091   99472.727273
chrysler         3519.384615  132366.863905
citroen          3756.072993  120160.583942
dacia            5897.736434   84728.682171
daewoo           1079.342105  121644.736842
daihatsu         1628.426230  115901.639344
fiat             2793.870048  117567.353407
ford             4022.905409  124349.689625
honda            4010.217054  123359.173127
hyundai          5371.792961  106718.426501
jaguar          11525.554054  124527.027027
jeep            11590.214953  127102.803738
kia              5889.298551  112739.130435
lada             2647.724138   85000.000000
lancia           3182.000000  123090.909091
land_rover      18934.272727  118333.333333
mazda            4059.059540  124871.447903
mercedes_benz    8526.623226  130848.387097
mini            10541.566986   8

We see that there is not exactly a correlation between mean price and mean mileage.

## Word Translation

While examining the data, we have seen that some of the columns have german language words. Let's map them to english.

In [9]:
autos
u_seller = autos["seller"].unique()
u_vehicle_type = autos["vehicle_type"].unique()
u_gearbox = autos["gearbox"].unique()
u_unrepaired_damage = autos["unrepaired_damage"].unique()
print(u_seller) #privat(private) , gewerblich(commercial)
print(u_vehicle_type) #bus (bus), limousine(limousine), kleinwagen(small car), kombi(station wagon)
#, nan,coupe(coupe), suv(suv), cabrio(cabrio), andere(other)
print(u_gearbox) # manuell(manual), automatik(automatic), nan
print(u_unrepaired_damage) #nein(no), nan, ja(yes)

['privat' 'gewerblich']
['bus' 'limousine' 'kleinwagen' 'kombi' nan 'coupe' 'suv' 'cabrio'
 'andere']
['manuell' 'automatik' nan]
['nein' nan 'ja']


We have investigated the german words and now we can create our dictionary for all german words.

In [12]:
ger_to_eng = {'privat':'private',
              'gewerblich':'commercial',
              'bus':'bus',
              'limousine':'limousine',
              'kleinwagen':'small car',
              'kombi':'station wagon',
              'coupe':'coupe',
              'suv':'suv',
              'cabrio':'cabrio',
              'andere':'other',
              'manuell':'manual',
              'automatik':'automatic',
              'nein':'no',
              'ja':'yes'
    
}
print(autos["seller"].unique())
autos['seller'] = autos['seller'].map(ger_to_eng)
print(autos["seller"].unique())

print(autos["vehicle_type"].unique())
autos['vehicle_type'] = autos['vehicle_type'].map(ger_to_eng)
print(autos["vehicle_type"].unique())

print(autos["gearbox"].unique())
autos['gearbox'] = autos['gearbox'].map(ger_to_eng)
print(autos["gearbox"].unique())

print(autos["unrepaired_damage"].unique())
autos['unrepaired_damage'] = autos['unrepaired_damage'].map(ger_to_eng)
print(autos["unrepaired_damage"].unique())

['privat' 'gewerblich']
['private' 'commercial']
['bus' 'limousine' 'kleinwagen' 'kombi' nan 'coupe' 'suv' 'cabrio'
 'andere']
['bus' 'limousine' 'small car' 'station wagon' nan 'coupe' 'suv' 'cabrio'
 'other']
['manuell' 'automatik' nan]
['manual' 'automatic' nan]
['nein' nan 'ja']
['no' nan 'yes']


## Date to Numeric Column

Previously we talked about that some of the date columns were in string format:
* date_crawled: added by the crawler
* last_seen: added by the crawler
* ad_created: from the website

By setting them such that they will contain only numeric character, we can set these columns to numerical columns.

In [17]:
autos["date_crawled"] = autos["date_crawled"].str[:10].str.replace("-","").astype(int)
autos["last_seen"] = autos["last_seen"].str[:10].str.replace("-","").astype(int)
autos["ad_created"] = autos["ad_created"].str[:10].str.replace("-","").astype(int)
autos[["date_crawled","last_seen","ad_created"]]

,date_crawled,last_seen,ad_created
0,20160326,20160406,20160326
1,20160404,20160406,20160404
2,20160326,20160406,20160326
3,20160312,20160315,20160312
4,20160401,20160401,20160401
5,20160321,20160406,20160321
6,20160320,20160323,20160320
7,20160316,20160407,20160316
8,20160322,20160326,20160322
9,20160316,20160406,20160316


## Name Column Analysis

In some cases, name columns may contain valuable information for us. For that reason, it would be beneficial if we have a look on the data that we have. In a case of additional information, we can use it and add an additional column to our data.

In [19]:
autos["name"] # it seems there is no logical structure in name as an info

0                         Peugeot_807_160_NAVTECH_ON_BOARD
1               BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik
2                               Volkswagen_Golf_1.6_United
3        Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...
4        Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...
5        Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...
6        VW_Golf_III_GT_Special_Electronic_Green_Metall...
7                                     Golf_IV_1.9_TDI_90PS
8                                               Seat_Arosa
9                Renault_Megane_Scenic_1.6e_RT_Klimaanlage
10                            VW_Golf_Tuning_in_siber/grau
11                              Mercedes_A140_Motorschaden
12       Smart_smart_fortwo_coupe_softouch_pure_MHD_Pan...
13                                      Audi_A3_1.6_tuning
14       Renault_Clio_3__Dynamique_1.2__16_V;_viele_Ver...
15             Corvette_C3_Coupe_T_Top_Crossfire_Injection
16                                     Opel_Vectra_B_Kom